In [ ]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2024-03-23 16:05:29
# @Last Modified: 2024-03-23 16:06:19
# ------------------------------------------------------------------------------ #
# Needs the data from `bayes_analysis.ipynb`
# ------------------------------------------------------------------------------ #

